In [31]:
# corona data
import pandas as pd
import seaborn as sb
import sklearn
import sklearn.linear_model
%matplotlib inline

cases_by_age = pd.read_csv('./files/corona/Cases_by_age.csv', sep=';')
cases_by_age.head()

,Aldersgruppe,Antal_bekræftede_COVID-19,Antal_testede,Procent_positive
0,0-9,2.252,222.688,"1,0"
1,10-19,6.908,394.170,"1,8"
2,20-29,9.478,467.258,"2,0"
3,30-39,6.758,400.277,"1,7"
4,40-49,7.419,413.284,"1,8"


In [89]:
cases_by_sex = pd.read_csv('./files/corona/Cases_by_sex.csv', sep=';', decimal=',')
cases_by_sex.head()

,Aldersgruppe,Kvinder_(procent),Mænd_(procent),I_alt
0,0-9,1.109 (49),1.143 (51),2.252
1,10-19,3.374 (49),3.534 (51),6.908
2,20-29,4.843 (51),4.635 (49),9.478
3,30-39,3.449 (51),3.309 (49),6.758
4,40-49,3.875 (52),3.544 (48),7.419


In [48]:
test_pos_over_time = pd.read_csv('./files/corona/Test_pos_over_time.csv', sep=';', thousands='.', decimal=',', index_col='Date')
test_pos_over_time_cleaned = test_pos_over_time[:-5]
#test_pos_over_time.head()
test_pos_over_time_cleaned.tail()
#test_pos_over_time_cleaned.plot(y='PosPct')
#test_pos_over_time_cleaned.plot(x='Tested_kumulativ', y='NewPositive')
#sb.pairplot(test_pos_over_time_cleaned)
#test_pos_over_time_cleaned.corr()

#xs = test_pos_over_time_cleaned['Tested_kumulativ']
#ys = data['NewPositive']

,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
Date,,,,,,
2020-10-26,1014,65341,1.6,544,65885,4982236
2020-10-27,1238,72448,1.7,625,73073,5055309
2020-10-28,1212,69510,1.7,581,70091,5125400
2020-10-29,1016,67013,1.5,644,67657,5193057
2020-10-30,1043,60246,1.7,562,60808,5253865


In [6]:
municipality_test_pos = pd.read_csv('./files/corona/Municipality_test_pos.csv', sep=';')
municipality_test_pos.head()

,Kommune_(id),Kommune_(navn),Antal_testede,Antal_bekræftede_COVID-19,Befolkningstal,Kumulativ_incidens_(per_100000)
0,101,København,362.641,8.744,632.340,1.383
1,147,Frederiksberg,58.654,1.377,104.305,1.320
2,151,Ballerup,28.163,689.000,48.602,1.418
3,153,Brøndby,20.589,740.000,35.090,2.109
4,155,Dragør,7.279,104.000,14.494,718.000


In [107]:
# build custom dataframe
total = 49594
column_names = ["Kommune", "Alder", "Køn", "Smittede %"]
my_dataframe = pd.DataFrame(columns=column_names)

data = []


df = pd.DataFrame(columns = column_names)
ages = cases_by_age['Aldersgruppe'][:-1]

women_data = cases_by_sex['Kvinder_(procent)'][:-1]
men_data = cases_by_sex['Mænd_(procent)'][:-1]

print(ages)
locations = municipality_test_pos['Kommune_(id)']
locations

for name in locations:
    for index, age in enumerate(ages):
        percent_woman = ( int(women_data[index].split('(')[0].strip().replace('.', '')) / total ) * 100
        percent_man = ( int(men_data[index].split('(')[0].strip().replace('.','')) / total ) * 100
        row_woman = {'Kommune': name, 'Alder': index+1, 'Køn': 1, "Smittede %": percent_woman}
        row_man = {'Kommune': name, 'Alder': index+1, 'Køn': 2, "Smittede %": percent_man}
        data.append(row_woman)
        data.append(row_man)
        
my_dataframe = my_dataframe.append(data, ignore_index=True)
        



0      0-9
1    10-19
2    20-29
3    30-39
4    40-49
5    50-59
6    60-69
7    70-79
8    80-89
9      90+
Name: Aldersgruppe, dtype: object


In [108]:
my_dataframe

,Kommune,Alder,Køn,Smittede %
0,101,1,1,2.236158
1,101,1,2,2.304714
2,101,2,1,6.803242
3,101,2,2,7.125862
4,101,3,1,9.765294
...,...,...,...,...
1975,860,8,2,2.677743
1976,860,9,1,1.508247
1977,860,9,2,1.193693
1978,860,10,1,0.572650


In [132]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# decision tree

y = my_dataframe['Smittede %']
x_features = ['Kommune', 'Alder', 'Køn']
X = my_dataframe[x_features]

corona_model = DecisionTreeRegressor(random_state=1)
corona_model.fit(X, y)
print(corona_model.predict(X.head()))
print(X.head())
print(corona_model.predict(pd.DataFrame([{'Kommune': 101, 'Alder': 1, 'Køn': 1}])))

predicted_corona = corona_model.predict(X)
print(mean_absolute_error(y, predicted_corona))



# split data into training and validation data, for both features and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
# Define model
corona_model_v2 = DecisionTreeRegressor()
# Fit model
corona_model_v2.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = corona_model_v2.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))



[2.2361576  2.30471428 6.80324233 7.125862   9.76529419]
  Kommune Alder Køn
0     101     1   1
1     101     1   2
2     101     2   1
3     101     2   2
4     101     3   1
[2.2361576]
5.691280779984709e-15
4.992527154877509e-15
